<a href="https://colab.research.google.com/github/jackjeanab/LangChain4Gemini/blob/main/4_6%E5%AF%A6%E4%BD%9CConditionalChain%E6%A9%9F%E5%88%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install --upgrade --quiet  langchain-google-genai

In [61]:
# 引入Chain模組
from langchain.chains import LLMChain

# 引入Gemini LLM模組
from langchain_google_genai import ChatGoogleGenerativeAI

# 引入prompt模組
from langchain_core.prompts import PromptTemplate

In [62]:
import os
from google.colab import userdata

# Get the API key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Initialize the Gemini Pro model
# Use "gemini-2.0-flash-exp"
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.5)

In [63]:
# 定義情緒分析的提示樣板
sentiment_analysis_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="根據這段話分析情緒，並僅回答 'positive' 或 'negative'：'{user_input}'"
)
# 建立情緒分析的 LLMChain
sentiment_analysis_chain = LLMChain(llm=llm, prompt=sentiment_analysis_prompt)

In [64]:
# 負面情緒應對的 PromptTemplate
negative_response_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="使用者說了這段話：'{user_input}'。請給出一段安撫的回應。"
)
negative_response_chain = LLMChain(llm=llm, prompt=negative_response_prompt)

In [65]:
# 正面情緒應對的 PromptTemplate
positive_response_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="使用者說了這段話：'{user_input}'。請給出一段正向互動的回應。"
)
positive_response_chain = LLMChain(llm=llm, prompt=positive_response_prompt)

In [66]:
def execute_conditional_chain(user_input):
    # 第一步：使用 LLM 來分析情緒
    sentiment_result = sentiment_analysis_chain.run({"user_input": user_input})

    # 第二步：根據情緒結果選擇要執行的chain
    if sentiment_result.strip().lower() == "negative":
        # 如果情緒為負面，執行負面應對chain
        return negative_response_chain.invoke({"user_input": user_input})
    else:
        # 如果情緒為正面，執行正面應對鏈結
        return positive_response_chain.invoke({"user_input": user_input})

In [67]:
# 執行 Conditional Chain
result = execute_conditional_chain("我對於你們的服務感到普通，服務人員表現普通。")

print(result["text"])

好的，對於您覺得我們的服務表現普通，我感到非常抱歉。我們非常重視每一位顧客的體驗，聽到您這樣的回饋，我們感到有些失望。

您的意見對我們來說非常重要，這能幫助我們不斷改進。我想了解更多關於您覺得「普通」的具體細節，例如：

*   **在哪些方面您覺得我們的服務表現普通？** (例如：回覆速度、服務態度、解決問題的能力等等)
*   **您期望的服務體驗是什麼樣的？**

如果您願意分享更多資訊，我們會非常感激。我們將會仔細檢視您的意見，並努力提升我們的服務品質。

同時，我想讓您知道，我們一直致力於提供更好的服務。我們希望未來能有機會讓您感受到我們的進步，並讓您對我們的服務感到滿意。

再次感謝您的回饋，您的意見對我們非常寶貴。
